In [12]:
import pandas as pd

In [13]:
pd.set_option('display.max_rows', None)

In [14]:
df = pd.read_csv('https://media.githubusercontent.com/media/mayurasandakalum/datasets/main/shop-customer-data/Customers.csv')
df.head()

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6


In [15]:
df.describe()

,CustomerID,Age,Annual Income ($),Spending Score (1-100),Work Experience,Family Size
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1000.500000,48.960000,110731.821500,50.962500,4.102500,3.768500
std,577.494589,28.429747,45739.536688,27.934661,3.922204,1.970749
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,500.750000,25.000000,74572.000000,28.000000,1.000000,2.000000
50%,1000.500000,48.000000,110045.000000,50.000000,3.000000,4.000000
75%,1500.250000,73.000000,149092.750000,75.000000,7.000000,5.000000
max,2000.000000,99.000000,189974.000000,100.000000,17.000000,9.000000


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              2000 non-null   int64 
 1   Gender                  2000 non-null   object
 2   Age                     2000 non-null   int64 
 3   Annual Income ($)       2000 non-null   int64 
 4   Spending Score (1-100)  2000 non-null   int64 
 5   Profession              1965 non-null   object
 6   Work Experience         2000 non-null   int64 
 7   Family Size             2000 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 125.1+ KB


remove null values

In [17]:
df.isnull().sum()

CustomerID                 0
Gender                     0
Age                        0
Annual Income ($)          0
Spending Score (1-100)     0
Profession                35
Work Experience            0
Family Size                0
dtype: int64

In [23]:
df = df.dropna(subset=['Profession'])
df.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income ($)         0
Spending Score (1-100)    0
Profession                0
Work Experience           0
Family Size               0
dtype: int64

In [41]:
df.columns.to_list()

['CustomerID',
 'Gender',
 'Age',
 'Annual Income ($)',
 'Spending Score (1-100)',
 'Profession',
 'Work Experience',
 'Family Size']

In [40]:
df.Age.value_counts().sort_index()

Age
0     23
1     25
2     15
3     18
4     15
5     20
6     14
7     18
8     22
9     12
10    11
11    13
12    20
13    25
14    19
15    19
16    21
17    17
18    23
19    24
20    27
21    23
22    17
23    23
24    19
25    24
26    16
27    19
28    19
29    20
30    26
31    31
32    30
33    20
34    27
35    20
36    21
37    22
38    24
39    18
40    23
41    24
42    11
43    23
44    15
45    17
46    18
47    23
48    20
49    19
50    14
51    12
52    29
53    19
54    28
55    17
56    18
57    14
58    23
59    23
60    22
61    11
62    23
63    27
64    24
65    21
66    16
67    23
68    23
69    15
70    19
71    12
72    18
73    14
74    12
75    21
76    15
77    12
78    23
79    17
80    19
81    17
82    17
83    24
84    26
85    21
86    16
87    20
88    15
89    22
90    19
91    28
92    19
93    18
94    18
95    19
96    22
97    22
98     9
99    16
Name: count, dtype: int64

In [34]:
for column in ["Gender", "Profession"]:
    print(column)
    print(df[column].value_counts())
    print("\n")

# df['Profession'].value_counts()

Gender
Gender
Female    1166
Male       799
Name: count, dtype: int64


Profession
Profession
Artist           612
Healthcare       339
Entertainment    234
Engineer         179
Doctor           161
Executive        153
Lawyer           142
Marketing         85
Homemaker         60
Name: count, dtype: int64


